In [ ]:
"""
Model
"""

import math
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Input, Activation, Dropout, Conv2D, Flatten, MaxPooling2D

from sklearn.model_selection import train_test_split